Code for dataLoader construction, training, and testing is largely referred from the complementary material from PyTorch course on Udacity. 

Link to the course: https://www.udacity.com/course/deep-learning-pytorch--ud188

### Load data

In [2]:
import numpy as np
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

y_train_valid -= 769
y_test -= 769

print(np.unique(y_train_valid))
print(np.unique(y_test))

[0 1 2 3]
[0 1 2 3]


### Shape of data

In [3]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


### Prepare Training Data

In [4]:
import torch
from torch.utils import data

X_train_valid = torch.from_numpy(X_train_valid.transpose(0,2,1)).float()
y_train_valid = torch.from_numpy(y_train_valid).long()
train_data = data.TensorDataset(X_train_valid, y_train_valid)

person_train_valid = torch.from_numpy(person_train_valid)
person_test = torch.from_numpy(person_test)

Data is transposed so that time dimension is before the electrode dimension.

In [5]:
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

batch_size = 128
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
train_loader = data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, drop_last=True)
valid_loader = data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, drop_last=True)

## RNN Architecture

In [6]:
import torch

train_on_gpu = torch.cuda.is_available()

# check if CUDA is available
device = torch.device("cuda:0" if train_on_gpu else "cpu")
print(device)
if not train_on_gpu:
    print('Training on CPU ...')
else:
    print('Training on GPU ...')

cuda:0
Training on GPU ...


In [11]:
import torch.nn as nn
import torch.nn.functional as F

class RNN(nn.Module):

    def __init__(self, num_classes=4):
        super(RNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv1d(22, 64, kernel_size=22, stride=2),
            nn.BatchNorm1d(64, affine=False),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool1d(kernel_size=12, stride=2),
            nn.Dropout(p=0.8),
            nn.Conv1d(64, 192, kernel_size=12),
            nn.BatchNorm1d(192, affine=False),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2),
            nn.Dropout(p=0.8),
            nn.Conv1d(192, 384, kernel_size=4, stride=2),
            nn.BatchNorm1d(384, affine=False),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool1d(kernel_size=4, stride=1),
        )
        self.rnn = nn.LSTM(input_size=22, hidden_size=15, dropout=0.8, num_layers=3, batch_first = True)
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.BatchNorm1d(15000),
            nn.LeakyReLU(inplace=True),
            nn.BatchNorm1d(15000),
            nn.LeakyReLU(inplace=True),
            nn.Linear(15000, num_classes),
        )

    def forward(self, x, hidden=None):
        #x = self.features(x)
        x, hidden = self.rnn(x, hidden)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x, hidden

Note that the convolution layers are commented in the forward().

In [12]:
# create a complete CNN
model = RNN()

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()

### Train the Model

In [16]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('./tensorboard/rnn')

In [9]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), weight_decay=0.01, lr=1e-3)

valid_loss_min = np.Inf # track change in validation loss

In [11]:
import time
# number of epochs to train the model
n_epochs = 5000
t0 = time.time()

hidden=None

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_loader:
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output, hidden = model(data, hidden)
        a, b = hidden
        hidden = (a.data, b.data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        output, _ = model(data)
        loss = criterion(output, target)
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    writer.add_scalar('Train/Loss', train_loss, epoch)
    writer.add_scalar('Valid/Loss', loss, epoch)

    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_EEG.pt')

time_total = time.time() - t0
print('Total time: {:4.3f}, average time per epoch: {:4.3f}'.format(time_total, time_total / n_epochs))

g Loss: 0.444252 	Validation Loss: 1.247299
Epoch: 4690 	Training Loss: 0.438310 	Validation Loss: 1.202509
Epoch: 4691 	Training Loss: 0.447957 	Validation Loss: 1.204568
Epoch: 4692 	Training Loss: 0.424285 	Validation Loss: 1.154306
Epoch: 4693 	Training Loss: 0.385227 	Validation Loss: 1.160632
Epoch: 4694 	Training Loss: 0.395918 	Validation Loss: 1.199911
Epoch: 4695 	Training Loss: 0.401770 	Validation Loss: 1.255857
Epoch: 4696 	Training Loss: 0.381088 	Validation Loss: 1.169661
Epoch: 4697 	Training Loss: 0.411731 	Validation Loss: 1.220967
Epoch: 4698 	Training Loss: 0.427690 	Validation Loss: 1.224593
Epoch: 4699 	Training Loss: 0.939139 	Validation Loss: 1.305815
Epoch: 4700 	Training Loss: 1.034862 	Validation Loss: 1.168285
Epoch: 4701 	Training Loss: 0.888939 	Validation Loss: 1.094032
Epoch: 4702 	Training Loss: 0.824923 	Validation Loss: 1.068267
Epoch: 4703 	Training Loss: 0.786585 	Validation Loss: 1.159292
Epoch: 4704 	Training Loss: 0.710284 	Validation Loss: 1.052

### Load the Model with the Lowest Validation Loss

In [13]:
model.load_state_dict(torch.load('model_EEG.pt'))

<All keys matched successfully>

### Test the Trained Network

In [11]:
from torch.utils import data

batch_size = 200

X_test = torch.from_numpy(X_test.transpose(0,2,1)).float()
y_test = torch.from_numpy(y_test).long()
test_data = data.TensorDataset(X_test, y_test)

test_loader = data.DataLoader(test_data, batch_size=batch_size)

In [12]:
# specify the target classes
classes = [769, 770, 771, 772]

# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(4))
class_total = list(0. for i in range(4))

model.eval()
for data, target in test_loader:
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    output, _ = model(data)
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    #print(pred)
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(correct.shape[0]):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(4):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.987191

Test Accuracy of   769: 65% (73/111)
Test Accuracy of   770: 51% (65/127)
Test Accuracy of   771: 50% (48/96)
Test Accuracy of   772: 69% (76/109)

Test Accuracy (Overall): 59% (262/443)
